In [1]:
!pip install --upgrade pip
!pip install langchain-community
!pip install bitsandbytes
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: la

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import AutoModel, pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [1]:
import os
path = os.getcwd()
train_path = os.path.join(path, "데이터셋", "train.csv")
test_path = os.path.join(path, "데이터셋", "test.csv")
test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)
train_df.head()

NameError: name 'pd' is not defined

In [4]:
def preprocess(df):
    df = df.copy()
    def 사고인지(x):
        return " ".join(x.split("-")[0:1])
    def 시간(x):
        return "".join(v.replace("정규작업", "").replace('휴일근무', "").replace('연장근무', "").replace('기타', "").strip('식사ㆍ휴식 등').
                        replace('작업 전', "").strip('출ㆍ퇴근 등').replace('불류불능', "").strip("식사ㆍ휴식").strip("-") for v in x.split(" "))
    def 시간1(x):
        return "".join(v.replace("작업전", "") for v in x.split(" "))
    def 시간2(x):
        return "".join(v.replace("오전", " AM ").replace("오후", " PM ") for v in x.split(" "))
    
    def 기온(x):
        return "".join(x.split(" "))[0:2]
    
    def 공사(x):
        return "".join(x.split("/")[0:1])
    
    def 종류(x):
        return "".join(x.split("/")[1:2])
    def 소분류(x):
        return "".join(x.split("/")[2:3])
    
    def 지상(x):
        return "".join(x.split(",")[0:1])
    
    def 지하(x):
        return "".join(x.split(",")[1:2])
    
    def 인적사고(x):
        return "".join(x.split("(")[0:1])
    
    def 원인(x):
        return "".join(x.split("(")[1:2])
    
    def 공종대(x):
        return "".join(x.split(">")[0:1])
    
    def 공종소(x):
        return "".join(x.split(">")[1:2])
    
    def 장소(x):
        return "".join(x.split(" ")[1:2])
    
    
    
        
    
    df["사고인지"] = df["사고인지 시간"].apply(사고인지)
    df["시간"] = df["사고인지 시간"].apply(시간)
    df["시간"] = df["시간"].apply(시간1)
    df["시간"] = df["시간"].apply(시간2)
    df["시간"] = pd.to_datetime(df["시간"], format = "%Y-%m-%d %p %I:%M")

    df["발생일시"] = df["발생일시"].apply(시간2)
    df["발생일시"] = df["발생일시"].apply(lambda x: x.replace("AM 00:", "AM 12:"))
    df["발생일시"] = pd.to_datetime(df["발생일시"], format = "%Y-%m-%d %p %I:%M")

    df["연도"] = df["발생일시"].dt.year
    df["월"] = df["발생일시"].dt.month
    df["일"] = df["발생일시"].dt.day
    df["주"] = df["발생일시"].dt.weekday
    df["시"] = df["발생일시"].dt.hour
    df["분"] = df["발생일시"].dt.minute

    df["기온"] = df["기온"].apply(lambda x: x.replace("℃", " "))
    df["기온"] = df["기온"].apply(lambda x: x.replace(" ", ""))
    df["기온"] = df["기온"].apply(lambda x: x.replace("0-", "-"))
    df.replace("", np.nan, inplace = True)
    df["기온"] = df["기온"].astype(float)
    df["기온"] = df["기온"].where((df["기온"] > -25) & (df["기온"] < 50), np.nan)
    df["기온"] = df["기온"].fillna(df["기온"].median())

    df["습도"] = df["습도"].apply(lambda x: x.replace("%", ""))
    df["습도"] = df["습도"].astype(float)
    df["습도"] = df["습도"].where((df["습도"] < 120), np.nan)
    df["습도"] = df["습도"].fillna(df["습도"].median())

    df["연면적"] = df["연면적"].apply(lambda x: x.replace("㎡", "").replace("-", "").replace(",", ""))
    df["연면적"].replace("", np.nan, inplace = True)
    df["연면적"] = df["연면적"].astype(float)
    df["연면적"] = df["연면적"].where((df["연면적"] > 0) & (df["연면적"] < 7e+5), np.nan)
    df["연면적"] = df["연면적"].fillna(df["연면적"].median())

    df["공사(대분류)"] = df["공사종류"].apply(공사)
    df["공사(중분류)"] = df["공사종류"].apply(종류)
    df["공사(소분류)"] = df["공사종류"].apply(소분류)

    df["층 정보"] = df["층 정보"].apply(lambda x: x.replace("-", "").strip("층").replace("지상", "").replace("지하", ""))
    df["층정보(지상)"] = df["층 정보"].apply(지상)
    df["층정보(지상)"] = df["층정보(지상)"].apply(lambda x: x.replace("층", ""))
    df["층정보(지하)"] = df["층 정보"].apply(지하)

    df.replace("", np.nan, inplace = True)

    df["층정보(지상)"] = df["층정보(지상)"].fillna(0)
    df["층정보(지하)"] = df["층정보(지하)"].fillna(0)
    
    df["인적사고"] = df["인적사고"].fillna("없음")
    df["인적사고(사고)"] = df["인적사고"].apply(인적사고)
    df["인적사고(원인)"] = df["인적사고"].apply(원인)
    df["인적사고(원인)"] = df["인적사고(원인)"].apply(lambda x: x.replace(")", ""))

    df["공종"] = df["공종"].fillna("모름")
    df["공종(대)"] = df["공종"].apply(공종대)
    df["공종(소)"] = df["공종"].apply(공종소)
    
    df["사고객체"] = df["사고객체"].fillna("모름")
    df["사고객체(대)"] = df["사고객체"].apply(공종대)
    df["사고객체(소)"] = df["사고객체"].apply(공종소)

    df["작업프로세스"] = df["작업프로세스"].fillna("기타")

    df["장소(대)"] = df["장소"].apply(공사)
    df["장소(소)"] = df["장소"].apply(종류)

    df["부위(대)"] = df["부위"].apply(공사)
    df["부위(소)"] = df["부위"].apply(종류)

    df["사고원인"] = df["사고원인"].fillna("모름")
    
    return df

train = preprocess(train_df)
test = preprocess(test_df)

train.head()

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
<ipython-input-4-da813b5a07f1>:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["연면적"].replace("", np.nan, inplace = True)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: i

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,...,인적사고(사고),인적사고(원인),공종(대),공종(소),사고객체(대),사고객체(소),장소(대),장소(소),부위(대),부위(소)
0,TRAIN_00000,2023-12-31 12:44:00,정규작업 -,맑음,1.0,30.0,건축 / 건축물 / 근린생활시설,4892.77,"14층, 3",떨어짐(5미터 이상 ~ 10미터 미만),...,떨어짐,5미터 이상 ~ 10미터 미만,건축,철근콘크리트공사,건설자재,철근,근린생활시설,내부,철근,고소
1,TRAIN_00001,2023-12-30 15:35:00,정규작업 -,맑음,10.0,90.0,토목 / 터널 / 철도터널,30908.00,NaN,끼임,...,끼임,,토목,터널공사,건설자재,볼트,철도터널,내부,볼트,바닥
2,TRAIN_00002,2023-12-30 14:30:00,정규작업 -,맑음,14.0,70.0,건축 / 건축물 / 업무시설,1994.62,"5층, 0",넘어짐(미끄러짐),...,넘어짐,미끄러짐,건축,철근콘크리트공사,기타,기타,업무시설,내부,기타,바닥
3,TRAIN_00003,2023-12-30 12:00:00,휴일근무 -,흐림,12.0,55.0,토목 / 하천 / 기타,30908.00,NaN,기타,...,기타,,토목,하천공사,부재,교각 기초,기타,동산교 신축구간,교각 기초,바닥
4,TRAIN_00004,2023-12-30 10:00:00,정규작업 -,맑음,0.0,10.0,건축 / 건축물 / 공동주택,59388.93,"27층, 3",넘어짐(미끄러짐),...,넘어짐,미끄러짐,건축,해체 및 철거공사,기타,건설폐기물,공동주택,내부,건설폐기물,바닥


In [5]:
training_data = train.apply(
    lambda x:{
        "question":(
            f"공사종류 대분류 '{x['공사(대분류)']}', 중분류 '{x['공사(중분류)']}', 소분류 '{x['공사(소분류)']}' 공사 중"
            f"공종 대분류 '{x['공종(대)']}', 소분류 '{x['공종(소)']}' 작업에서"
            f"사고 객체'{x['사고객체(대)']}'(소분류 '{x['사고객체(소)']}') 와 같은 사고가 발생했습니다."
            f"작업 프로세스는 '{x['작업프로세스']}'이며 사고 원인은 '{x['사고원인']}' 입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer":x["재발방지대책 및 향후조치계획"]
    },
    axis = 1
)

training_data = pd.DataFrame(list(training_data))

In [6]:
testing_data = test.apply(
    lambda x:{
        "question":(
            f"공사종류 대분류 '{x['공사(대분류)']}', 중분류 '{x['공사(중분류)']}', 소분류 '{x['공사(소분류)']}' 공사 중"
            f"공종 대분류 '{x['공종(대)']}', 소분류 '{x['공종(소)']}' 작업에서"
            f"사고 객체'{x['사고객체(대)']}'(소분류 '{x['사고객체(소)']}') 와 같은 사고가 발생했습니다."
            f"작업 프로세스는 '{x['작업프로세스']}'이며 사고 원인은 '{x['사고원인']}' 입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis = 1
)

testing_data = pd.DataFrame(list(testing_data))

testing_data.head()

,question
0,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 교정 및 군사시설' ..."
1,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 운수시설' 공사 중공..."
2,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 공동주택' 공사 중공..."
3,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 공동주택' 공사 중공..."
4,"공사종류 대분류 '토목 ', 중분류 ' 교량 ', 소분류 ' 도로교량' 공사 중공종..."


In [7]:
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [8]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map = "auto")

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
!pip install wandb

In [10]:
!wandb login 7b35707cdec15b90dd36fbb1f675d78473958ac4

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
training_data = training_data.reset_index(drop=True)

In [12]:
training_data.iloc[14582]

question    공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 공동주택' 공사 중공...
answer      리프트 결함 유무 확인 및 교체와 사고 예방을 위한 특별 안전 교육 실시와 향후 결...
Name: 14582, dtype: object

In [13]:
training_data.head()

,question,answer
0,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 근린생활시설' 공사 ...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
1,"공사종류 대분류 '토목 ', 중분류 ' 터널 ', 소분류 ' 철도터널' 공사 중공종...",재발 방지 대책 마련과 안전교육 실시.
2,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 업무시설' 공사 중공...",현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조...
3,"공사종류 대분류 '토목 ', 중분류 ' 하천 ', 소분류 ' 기타' 공사 중공종 대...","위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, ..."
4,"공사종류 대분류 '건축 ', 중분류 ' 건축물 ', 소분류 ' 공동주택' 공사 중공...",자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 ...


In [14]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(training_data)  # pandas.DataFrame -> datasets.Dataset

In [15]:
def preprocess_function(examples):
    # 예시에서 question과 answer를 모델 입력에 맞게 변환
    return tokenizer(examples['question'], examples['answer'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/23422 [00:00<?, ? examples/s]

In [16]:
from peft import LoraConfig, get_peft_model,  prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer

import os
os.makedirs("/kaggle/working/Lora_model", exist_ok = True)

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    target_modules = ["q_proj", "v_proj"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM",
)

trained_model = prepare_model_for_kbit_training(model)
models = get_peft_model(trained_model, lora_config)


import wandb
from transformers import TrainerCallback
wandb.init(project="Lora_FineTuning_Experiment")

class WandBBatchCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, **kwargs):
        current_step = state.global_step
        batch_size = args.per_device_train_batch_size * args.gradient_accumulation_steps
        wandb.log({"Current Step": current_step, "Batch Size": batch_size})


training_args = TrainingArguments(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    warmup_steps = 100,
    learning_rate = 2e-4,
    num_train_epochs = 3,
    save_steps = 500,
    output_dir = "./Lora_model",
    logging_dir = "./logs",
    run_name="Lora_FineTuning_Experiment"
)

trainer = Trainer(
    model = models,
    args = training_args,
    train_dataset = train_dataset
)

trainer.train()

model.save_pretrained("/kaggle/working/Lora_model")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yunjun0914 (yunjun0914-google). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.91 GiB. GPU 0 has a total capacity of 15.89 GiB of which 3.04 GiB is free. Process 3014 has 12.84 GiB memory in use. Of the allocated memory 11.80 GiB is allocated by PyTorch, and 776.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)